In [3]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from functools import reduce
# from intervaltree import Interval, IntervalTree
# from dependencies.GDSC_methods import filter_protein_coding, filter_mini_cancer, is_probe_in_tree, create_tree

raw_folder = '/DATA/s.mourragui/data/2020_10_GDSC_data/'
output_folder = '/DATA/s.mourragui/data/2020_10_GDSC_data'

In [4]:
# Read models
model_file = '%sraw/methSampleId_2_cosmicIds.xlsx'%(raw_folder)
model_df = pd.read_excel(model_file)
model_df = model_df[model_df['Cell_Line'] == 'YES']
model_df['METH_ID'] = ['%s_%s'%(e,f)
                       for e,f in zip(model_df['Sentrix_ID'], model_df['Sentrix_Position'])]
model_df = model_df[['METH_ID', 'Tissue', 'Sample_Name']]

FileNotFoundError: [Errno 2] No such file or directory: '/DATA/s.mourragui/data/2020_10_GDSC_data/raw/methSampleId_2_cosmicIds.xlsx'

In [6]:
methylation_file = '%sraw/F2_METH_CELL_DATA.txt'%(raw_folder)
meth_df = pd.read_csv(methylation_file, sep='\t', index_col=0)
meth_df.shape

(14726, 1029)

In [7]:
meth_df = meth_df.T.merge(model_df.set_index('METH_ID'),
                          left_index=True,
                          right_index=True,
                          how='left')
meth_df = meth_df.groupby(['Sample_Name', 'Tissue']).agg('mean')

In [8]:
meth_df.index = meth_df.index.rename(['model_name', 'tissue'])
# meth_df.index = meth_df.index.set_levels([meth_df.index.levels[0].astype(str), 
#                                         meth_df.index.levels[-1]])
meth_df.index = pd.MultiIndex.from_tuples([(str(x[0]), str(x[1]).upper()) for x in meth_df.index])

In [9]:
meth_df

,,chr1:10003165-10003585,chr1:100315420-100316009,chr1:100435297-100436070,chr1:100503482-100504404,chr1:10057121-10058108,chr1:100598130-100598754,chr1:100731632-100731999,chr1:100817709-100818899,chr1:10092532-10093404,chr1:101004471-101005885,...,chr9:99179479-99181723,chr9:99212164-99213507,chr9:99328810-99329764,chr9:99380814-99382236,chr9:99416904-99417592,chr9:99540025-99540472,chr9:99616401-99616940,chr9:99775098-99776102,chr9:99800856-99801724,chr9:99962136-99962426
697,HAEMATOPOIETIC AND LYMPHOID TISSUE,0.298852,0.247581,0.544867,0.253991,0.897269,0.319829,0.351015,0.160873,0.347642,0.850638,...,0.182682,0.445366,0.441266,0.870162,0.693336,0.362607,0.494730,0.356146,0.841023,0.493052
5637,URINARY TRACT,0.207451,0.213395,0.255311,0.234563,0.214327,0.272923,0.264034,0.101377,0.257876,0.677746,...,0.133019,0.160474,0.124177,0.261473,0.590505,0.213558,0.330050,0.331833,0.305816,0.314826
22RV1,PROSTATE,0.167531,0.176802,0.246339,0.249161,0.435564,0.256040,0.272493,0.116850,0.189623,0.816168,...,0.167701,0.230101,0.320823,0.409860,0.547167,0.252352,0.534457,0.336614,0.138241,0.212447
23132-87,STOMACH,0.147465,0.164950,0.115914,0.182537,0.433117,0.231231,0.226982,0.211016,0.177874,0.440325,...,0.158631,0.127167,0.091057,0.237301,0.433755,0.223517,0.312370,0.303954,0.126140,0.159335
42-MG-BA,CENTRAL NERVOUS SYSTEM,0.202527,0.247592,0.316943,0.222415,0.556366,0.278966,0.251533,0.188752,0.271642,0.708942,...,0.150712,0.121277,0.099095,0.378843,0.536608,0.243218,0.317670,0.303063,0.336228,0.175628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YH-13,CENTRAL NERVOUS SYSTEM,0.207342,0.167199,0.193633,0.189009,0.336063,0.279951,0.213661,0.105031,0.226564,0.398779,...,0.111503,0.118645,0.133531,0.279817,0.463680,0.218868,0.332607,0.242554,0.318765,0.161824
YKG-1,CENTRAL NERVOUS SYSTEM,0.201779,0.221038,0.274851,0.233499,0.324314,0.280434,0.243951,0.122805,0.246760,0.298615,...,0.152071,0.147067,0.116751,0.317423,0.495610,0.230390,0.277490,0.315399,0.300994,0.094842
YMB-1-E,BREAST,0.200625,0.152257,0.151656,0.249544,0.664511,0.269360,0.252894,0.251060,0.248002,0.750564,...,0.154494,0.268131,0.311740,0.337513,0.545872,0.668073,0.339896,0.332781,0.316893,0.276506
YT,HAEMATOPOIETIC AND LYMPHOID TISSUE,0.332950,0.313973,0.385712,0.254299,0.890986,0.309745,0.334015,0.382517,0.327898,0.679910,...,0.186265,0.365751,0.865715,0.866165,0.853114,0.742806,0.487059,0.475000,0.807618,0.563769


In [10]:
genes_lookup_file = '%s/pybiomart_gene_status.csv'%(raw_folder)
genes_lookup_df = pd.read_csv(genes_lookup_file, sep='\t', index_col=0)
pc_interval_trees = create_tree(meth_df, genes_lookup_df[genes_lookup_df.status == 'protein_coding'])
protein_coding_probes = []
for pos in meth_df.columns:
    if len(is_probe_in_tree(pos, pc_interval_trees)) >= 1:
        protein_coding_probes.append(pos)
meth_pc_df = meth_df[protein_coding_probes]

In [11]:
meth_pc_df

,,chr1:10003165-10003585,chr1:100435297-100436070,chr1:100503482-100504404,chr1:10057121-10058108,chr1:100731632-100731999,chr1:10092532-10093404,chr1:101004471-101005885,chr1:10270015-10271306,chr1:1027509-1027730,chr1:10458548-10459997,...,chr9:96108466-96108992,chr9:96338040-96339836,chr9:97488818-97489172,chr9:976162-977828,chr9:98075418-98075741,chr9:98079189-98080213,chr9:98637245-98638445,chr9:98783216-98784364,chr9:99145524-99145849,chr9:99962136-99962426
697,HAEMATOPOIETIC AND LYMPHOID TISSUE,0.298852,0.544867,0.253991,0.897269,0.351015,0.347642,0.850638,0.334110,0.915966,0.266004,...,0.914388,0.360911,0.418658,0.798963,0.707034,0.568519,0.272947,0.822431,0.913653,0.493052
5637,URINARY TRACT,0.207451,0.255311,0.234563,0.214327,0.264034,0.257876,0.677746,0.171207,0.923701,0.134679,...,0.752930,0.116214,0.238813,0.396512,0.623581,0.547536,0.177449,0.429776,0.319359,0.314826
22RV1,PROSTATE,0.167531,0.246339,0.249161,0.435564,0.272493,0.189623,0.816168,0.220325,0.915929,0.154591,...,0.857770,0.506050,0.251346,0.299293,0.669400,0.448721,0.239537,0.561328,0.626794,0.212447
23132-87,STOMACH,0.147465,0.115914,0.182537,0.433117,0.226982,0.177874,0.440325,0.165262,0.909131,0.143686,...,0.647452,0.271127,0.210230,0.492569,0.628751,0.460081,0.212215,0.761538,0.509242,0.159335
42-MG-BA,CENTRAL NERVOUS SYSTEM,0.202527,0.316943,0.222415,0.556366,0.251533,0.271642,0.708942,0.150256,0.921378,0.143861,...,0.642129,0.156881,0.248771,0.596610,0.720490,0.546806,0.157562,0.532229,0.827324,0.175628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YH-13,CENTRAL NERVOUS SYSTEM,0.207342,0.193633,0.189009,0.336063,0.213661,0.226564,0.398779,0.136854,0.773743,0.177244,...,0.227682,0.111785,0.286860,0.569720,0.692199,0.526544,0.156885,0.435184,0.495066,0.161824
YKG-1,CENTRAL NERVOUS SYSTEM,0.201779,0.274851,0.233499,0.324314,0.243951,0.246760,0.298615,0.168089,0.890057,0.097430,...,0.545528,0.128992,0.199272,0.455655,0.734416,0.521433,0.150185,0.329597,0.489748,0.094842
YMB-1-E,BREAST,0.200625,0.151656,0.249544,0.664511,0.252894,0.248002,0.750564,0.225442,0.923073,0.111227,...,0.753481,0.239235,0.286634,0.522079,0.703877,0.534037,0.200564,0.882380,0.718792,0.276506
YT,HAEMATOPOIETIC AND LYMPHOID TISSUE,0.332950,0.385712,0.254299,0.890986,0.334015,0.327898,0.679910,0.373866,0.853936,0.264993,...,0.905820,0.455651,0.335517,0.858433,0.870067,0.571947,0.237683,0.895019,0.907677,0.563769


In [15]:
pc_file = '/DATA/s.mourragui/data/2020_10_GDSC_data_mini_cancer/methylation/all_methylation_data.pkl'
all_file = '/DATA/s.mourragui/data/2020_10_GDSC_data_mini_cancer/methylation_all/all_methylation_all_data.pkl'

pc_df = pd.read_pickle(pc_file, compression='gzip')
all_df = pd.read_pickle(all_file, compression='gzip')

In [16]:
pc_df.head()

,,chr1:100435297-100436070,chr1:100503482-100504404,chr1:10270015-10271306,chr1:10458548-10459997,chr1:107599205-107600410,chr1:107682889-107684463,chr1:11119030-11120634,chr1:11159387-11160136,chr1:111682227-111683303,chr1:112524468-112525385,...,chr9:6757336-6758872,chr9:74764241-74764903,chr9:8857485-8858708,chr9:91605764-91606141,chr9:91606335-91606823,chr9:91615900-91616245,chr9:91792662-91793611,chr9:91925372-91926639,chr9:91933284-91933907,chr9:99145524-99145849
697,HAEMATOPOIETIC AND LYMPHOID TISSUE,0.544867,0.253991,0.334110,0.266004,0.453127,0.654512,0.383042,0.313846,0.218677,0.786200,...,0.219442,0.918074,0.490343,0.652619,0.885537,0.851897,0.715697,0.238707,0.279628,0.913653
5637,URINARY TRACT,0.255311,0.234563,0.171207,0.134679,0.360781,0.164495,0.276517,0.288703,0.180840,0.563749,...,0.222706,0.316189,0.324610,0.639188,0.883096,0.841424,0.193593,0.178038,0.232738,0.319359
22RV1,PROSTATE,0.246339,0.249161,0.220325,0.154591,0.439495,0.397331,0.310896,0.191753,0.225880,0.343992,...,0.216376,0.801893,0.292035,0.643947,0.852335,0.704451,0.449876,0.212031,0.223960,0.626794
23132-87,STOMACH,0.115914,0.182537,0.165262,0.143686,0.408586,0.548346,0.225311,0.144299,0.221427,0.364220,...,0.189705,0.298480,0.529648,0.465534,0.529777,0.387724,0.633514,0.152580,0.219462,0.509242
42-MG-BA,CENTRAL NERVOUS SYSTEM,0.316943,0.222415,0.150256,0.143861,0.404943,0.177697,0.306376,0.303783,0.160306,0.574752,...,0.211161,0.658629,0.418591,0.797158,0.656684,0.875369,0.266363,0.207041,0.293016,0.827324


In [17]:
all_df.head()

,,chr1:10003165-10003585,chr1:100315420-100316009,chr1:100435297-100436070,chr1:100503482-100504404,chr1:10057121-10058108,chr1:100598130-100598754,chr1:100731632-100731999,chr1:100817709-100818899,chr1:10092532-10093404,chr1:101004471-101005885,...,chr9:99179479-99181723,chr9:99212164-99213507,chr9:99328810-99329764,chr9:99380814-99382236,chr9:99416904-99417592,chr9:99540025-99540472,chr9:99616401-99616940,chr9:99775098-99776102,chr9:99800856-99801724,chr9:99962136-99962426
697,HAEMATOPOIETIC AND LYMPHOID TISSUE,0.298852,0.247581,0.544867,0.253991,0.897269,0.319829,0.351015,0.160873,0.347642,0.850638,...,0.182682,0.445366,0.441266,0.870162,0.693336,0.362607,0.494730,0.356146,0.841023,0.493052
5637,URINARY TRACT,0.207451,0.213395,0.255311,0.234563,0.214327,0.272923,0.264034,0.101377,0.257876,0.677746,...,0.133019,0.160474,0.124177,0.261473,0.590505,0.213558,0.330050,0.331833,0.305816,0.314826
22RV1,PROSTATE,0.167531,0.176802,0.246339,0.249161,0.435564,0.256040,0.272493,0.116850,0.189623,0.816168,...,0.167701,0.230101,0.320823,0.409860,0.547167,0.252352,0.534457,0.336614,0.138241,0.212447
23132-87,STOMACH,0.147465,0.164950,0.115914,0.182537,0.433117,0.231231,0.226982,0.211016,0.177874,0.440325,...,0.158631,0.127167,0.091057,0.237301,0.433755,0.223517,0.312370,0.303954,0.126140,0.159335
42-MG-BA,CENTRAL NERVOUS SYSTEM,0.202527,0.247592,0.316943,0.222415,0.556366,0.278966,0.251533,0.188752,0.271642,0.708942,...,0.150712,0.121277,0.099095,0.378843,0.536608,0.243218,0.317670,0.303063,0.336228,0.175628


# Mutations

In [24]:
raw_folder = '/DATA/s.mourragui/data/2020_10_GDSC_data/'
output_folder = '/DATA/s.mourragui/data/2020_10_GDSC_data'
mut_file = '%sraw/mutations_20191101.csv'%(raw_folder)
df = pd.read_csv(mut_file, index_col=None, header=[0,1,2])

model_file = '%smodel_list_20191104.csv'%(raw_folder)
model_df = pd.read_csv(model_file)

In [25]:
df.columns = df.columns.droplevel(1).droplevel(1)
df.head()

,gene_id,gene_symbol,model_id,protein_mutation,rna_mutation,cdna_mutation,cancer_driver,model_name
0,SIDG33650,SAMD11,SIDM01293,p.Q413fs*38,r.1314delC,c.1231delC,False,COLO_099
1,SIDG22024,NOC2L,SIDM01293,-,r.?,-,False,COLO_099
2,SIDG22024,NOC2L,SIDM01293,-,r.?,-,False,COLO_099
3,SIDG25014,PLEKHN1,SIDM01293,-,r.?,-,False,COLO_099
4,SIDG11201,HES4,SIDM01293,-,r.?,-,False,COLO_099


In [26]:
df = df[['model_id', 'gene_symbol']]
df['INDIC'] = 1

In [27]:
df = df.pivot_table(columns=['gene_symbol'], index='model_id', fill_value=0)
df.head()

INDIC                                                       ...  \
gene_symbol  A1BG A1CF A2M A2ML1 A3GALT2 A4GALT A4GNT AAAS AACS AACSP1  ...   
model_id                                                                ...   
SIDM00003       0    0   0     0       0      0     0    0    1      0  ...   
SIDM00023       0    0   0     0       0      0     0    0    0      0  ...   
SIDM00040       0    0   1     0       0      0     0    0    0      0  ...   
SIDM00041       0    0   0     0       0      0     0    0    0      0  ...   
SIDM00042       0    0   0     0       0      0     0    0    0      0  ...   

                                                                      
gene_symbol ZWILCH ZWINT ZXDA ZXDB ZXDC ZYG11A ZYG11B ZYX ZZEF1 ZZZ3  
model_id                                                              
SIDM00003        0     0    0    0    0      0      0   0     0    0  
SIDM00023        0     0    0    0    0      0      0   0     0    0  
SIDM00040        1     0    0    0    0      0      0   0     0    0  
SIDM00041        0     0    0    0    0      0      0   0     0    1  
SIDM00042        0     0    0    0    0      0      0   0     0    0  

[5 rows x 21972 columns]

In [28]:
df.columns = df.columns.droplevel(0)

In [29]:
df = df.merge(model_df[['model_name', 'model_id', 'tissue']].set_index('model_id'),
                  how='left',
                  left_index=True,
                  right_index=True)

In [31]:
df = df.reset_index(drop=True).set_index(['model_name', 'tissue'])
df = df.dropna(axis=1)
df.index = pd.MultiIndex.from_tuples([(x[0], str(x[1]).upper()) for x in df.index])

In [32]:
df

,,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AACSP1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
M14,SKIN,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
TE-12,ESOPHAGUS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TMK-1,STOMACH,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
STS-0421,STOMACH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
PL4,PANCREAS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HCM-SANG-0299-C15,ESOPHAGUS,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HCM-SANG-0299-C15-B,ESOPHAGUS,0,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
HCM-SANG-0309-C15,ESOPHAGUS,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
HCM-SANG-0285-C18,LARGE INTESTINE,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [91]:
df.columns = df.columns.droplevel(1).droplevel(1)

,model_id,SIDM00001,SIDM00002,SIDM00003,SIDM00005,SIDM00006,SIDM00007,SIDM00008,SIDM00009,SIDM00011,SIDM00014,...,SIDM01242,SIDM01245,SIDM01246,SIDM01247,SIDM01248,SIDM01251,SIDM01254,SIDM01257,SIDM01259,SIDM01265
gene_id,symbol,,,,,,,,,,,,,,,,,,,,,
SIDG00001,A1BG,37.0,152.0,124.0,149.0,121.0,357.0,0.0,127.0,43.0,22.0,...,398.0,273.0,191.0,274.0,249.0,92.0,24.0,47.0,401.0,1.0
SIDG00002,A1BG-AS1,121.0,799.0,2038.0,244.0,1167.0,2847.0,2.0,1886.0,372.0,2.0,...,2590.0,1431.0,1433.0,2848.0,1989.0,18.0,116.0,4.0,2373.0,0.0
SIDG00003,A1CF,3.0,0.0,0.0,1.0,11.0,3.0,8.0,3.0,4.0,350.0,...,2.0,0.0,0.0,1.0,1.0,1.0,0.0,10373.0,1.0,15.0
SIDG00004,A2M,38.0,230.0,11771.0,64.0,30467.0,27.0,5.0,17.0,32961.0,42.0,...,10310.0,0.0,1528.0,1.0,22.0,2.0,3.0,15202.0,0.0,12.0
SIDG00005,A2M-AS1,0.0,75.0,117.0,139.0,128.0,231.0,2.0,8.0,188.0,42.0,...,45.0,5.0,29.0,65.0,83.0,11.0,1.0,87.0,34.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIDG42471,ZYG11B,1320.0,12279.0,15504.0,1389.0,12647.0,7974.0,1143.0,4523.0,1542.0,8314.0,...,13682.0,8234.0,6411.0,7883.0,31788.0,5852.0,1142.0,7311.0,11530.0,1871.0
SIDG42472,ZYX,1735.0,9299.0,39582.0,1025.0,101402.0,60313.0,6514.0,13832.0,7935.0,47151.0,...,19537.0,3001.0,28917.0,1.0,39777.0,26774.0,233.0,12479.0,5886.0,2316.0
SIDG42473,ZYXP1,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0


In [92]:
df.index = df.index.droplevel(0)
df = df.T
df = df.dropna(axis=1)

In [93]:
df

symbol,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2ML1-AS2,A2MP1,A3GALT2,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZZEF1,ZZZ3
model_id,,,,,,,,,,,,,,,,,,,,,
SIDM00001,37.0,121.0,3.0,38.0,0.0,1.0,0.0,21.0,1.0,0.0,...,10247.0,140.0,807.0,1174.0,239.0,0.0,1320.0,1735.0,3306.0,2273.0
SIDM00002,152.0,799.0,0.0,230.0,75.0,193.0,1.0,2.0,1.0,51.0,...,12278.0,934.0,3368.0,19136.0,458.0,0.0,12279.0,9299.0,17917.0,19371.0
SIDM00003,124.0,2038.0,0.0,11771.0,117.0,2.0,0.0,0.0,8.0,20.0,...,41688.0,607.0,2595.0,7937.0,3940.0,17.0,15504.0,39582.0,25356.0,25481.0
SIDM00005,149.0,244.0,1.0,64.0,139.0,17.0,3.0,40.0,3.0,6.0,...,7344.0,287.0,662.0,2373.0,203.0,0.0,1389.0,1025.0,2295.0,2743.0
SIDM00006,121.0,1167.0,11.0,30467.0,128.0,4.0,1.0,0.0,13.0,34.0,...,14348.0,715.0,2705.0,6488.0,4330.0,0.0,12647.0,101402.0,11354.0,16203.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIDM01251,92.0,18.0,1.0,2.0,11.0,8.0,2.0,1.0,1.0,1.0,...,23535.0,2208.0,7801.0,12946.0,4169.0,0.0,5852.0,26774.0,52040.0,9406.0
SIDM01254,24.0,116.0,0.0,3.0,1.0,0.0,0.0,11.0,0.0,0.0,...,9029.0,1.0,360.0,1389.0,3.0,0.0,1142.0,233.0,9158.0,3401.0
SIDM01257,47.0,4.0,10373.0,15202.0,87.0,20.0,2.0,2.0,0.0,7.0,...,12649.0,10.0,2405.0,10773.0,2250.0,0.0,7311.0,12479.0,6175.0,14599.0


In [83]:
cnv_df = cnv_df.merge(model_df[['model_id', 'tissue', 'model_name']].set_index('model_id'),
                          how='left',
                          left_index=True,
                          right_index=True)

In [84]:
cnv_df = cnv_df.reset_index(drop=True)

In [85]:
cnv_df = cnv_df.set_index(['model_name', 'tissue'])
cnv_df = cnv_df.dropna(axis=1)
cnv_df.index = pd.MultiIndex.from_tuples([(x[0], str(x[1]).upper()) for x in cnv_df.index])

In [86]:
cnv_df

,,A1BG,A1BG-AS1,A1CF,A2M,A2ML1,A2MP1,A4GALT,A4GNT,AAAS,AACS,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
M14,SKIN,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,-1.0,0.0,1.0,-1.0,1.0,1.0,1.0,2.0,2.0,2.0
TE-12,ESOPHAGUS,-1.0,-1.0,0.0,1.0,1.0,1.0,-1.0,0.0,0.0,0.0,...,-1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
TMK-1,STOMACH,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,...,-1.0,1.0,0.0,1.0,-2.0,0.0,0.0,2.0,-1.0,1.0
STS-0421,STOMACH,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-1.0,-1.0,0.0,1.0,1.0,0.0,0.0,2.0,-1.0,0.0
PL4,PANCREAS,1.0,1.0,-1.0,1.0,1.0,1.0,0.0,1.0,-1.0,-1.0,...,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CGTH-W-1,THYROID,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,-1.0,-1.0,...,0.0,1.0,-1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
H9,HAEMATOPOIETIC AND LYMPHOID,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,-1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
GR-ST,HAEMATOPOIETIC AND LYMPHOID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,-1.0,1.0,0.0,0.0,1.0,0.0,0.0
YMB-1-E,BREAST,0.0,0.0,0.0,1.0,1.0,1.0,-1.0,0.0,0.0,0.0,...,-2.0,-1.0,1.0,0.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0
